## 1. Similarity

In [20]:
from 

v_1 = np.array([1,2,3,4,5])
v_2 = np.array([5,4,3,2,1])
v_3 = np.array([2,3,4,5,6])
v_4 = np.array([11,19,28,32,47])

In [22]:
# Euclidean Distance Similarity

np.sqrt(sum((v_1 - v_2)**2)), np.linalg.norm(v_1 - v_2)

(6.324555320336759, 6.324555320336759)

In [39]:
# Cosine Similarity

# 분자 - 벡터의 내적
sum(v_1*v_2), np.dot(v_1,v_2)

# 분모 - 벡터의 크기
num2 = np.sqrt(sum(v_1*v_1)) * np.sqrt(sum(v_2*v_2))

# 결과
np.dot(v_1,v_2)/num2, 1 - sp.spatial.distance.cosine(v_1,v_2)

(0.6363636363636364, 0.6363636363636364)

In [40]:
np.linalg.norm(v_1 - v_2),\
np.linalg.norm(v_1 - v_3),\
np.linalg.norm(v_1 - v_4)

(6.324555320336759, 2.23606797749979, 59.682493245507096)

In [41]:
1-sp.spatial.distance.cosine(v_1,v_2),\
1-sp.spatial.distance.cosine(v_1,v_3),\
1-sp.spatial.distance.cosine(v_1,v_4)

(0.6363636363636364, 0.9949366763261821, 0.9971083087048903)

In [49]:
columns = ["art_1","art_2","art_3","art_4","art_5"]
index = ["u_1","u_2","u_3","u_4"]

data = np.array([[5,3,0,0,2],
                [2,0,0,1,4],
                [0,0,4,3,1],
                [4,0,4,5,0],])

s_df = pd.DataFrame(data, columns=columns, index=index)
s_df

,art_1,art_2,art_3,art_4,art_5
u_1,5,3,0,0,2
u_2,2,0,0,1,4
u_3,0,0,4,3,1
u_4,4,0,4,5,0


In [67]:
def cosine_similarity(v_1,v_2):
    
#     print(list(v_1),list(v_2))
    idx = v_1.nonzero()[0]
#     print(idx)
    v_1 = np.array(v_1)[idx]
    v_2 = np.array(v_2)[idx]
#     print(v_1,v_2)
    
    return 1-sp.spatial.distance.cosine(v_1,v_2)

In [68]:
# test code - cosine similarity

cosine_similarity(s_df.loc["u_1"],s_df.loc["u_2"])

0.6529286250990105

In [69]:
def similarity_matrix(s_df, similarity_func):
    
    index = s_df.index
    
    df = s_df.T
    
    matrix = []
    for idx_1, value_1 in df.items():
        row=[]
        for idx_2, value_2 in df.items():
            row.append(similarity_func(value_1, value_2))
        matrix.append(row)
        
    return pd.DataFrame(matrix, index=index, columns=index)
            

In [74]:
sm_df = similarity_matrix(s_df, cosine_similarity)
sm_df

,u_1,u_2,u_3,u_4
u_1,1.000000,0.652929,0.324443,0.811107
u_2,0.729397,1.000000,0.483046,0.443039
u_3,0.196116,0.332956,1.000000,0.949474
u_4,0.529813,0.770054,0.821210,1.000000


In [90]:
user, closer_count = "u_1", 2

ms_df = sm_df.drop(user)
ms_df = ms_df.sort_values(user, ascending=False)
ms_df = ms_df[:closer_count]

ms_df 

s_df.loc[ms_df.index]

,art_1,art_2,art_3,art_4,art_5
u_2,2,0,0,1,4
u_4,4,0,4,5,0


In [94]:
mean = np.zeros(len(s_df.columns))
for ms_user, sms_value in ms_df[user].items():
    mean += s_df.loc[ms_user]
mean /= len(ms_df[user])
mean

art_1    3.0
art_2    0.0
art_3    2.0
art_4    3.0
art_5    2.0
Name: u_2, dtype: float64

In [102]:
pre_df = pd.DataFrame(columns=s_df.columns)
pre_df.loc["user"] = s_df.loc[user]
pre_df.loc["mean"] = mean
pre_df

,art_1,art_2,art_3,art_4,art_5
user,5,3,0,0,2
mean,3,0,2,3,2


In [127]:
def mean_score(s_df, sm_df, target, closer_count):
    
    ms_df = sm_df.drop(user)
    ms_df = ms_df.sort_values(user, ascending=False)
    ms_df = ms_df[:closer_count]
    
    ms_df = s_df.loc[ms_df.index]
    
    pre_df = pd.DataFrame(columns=s_df.columns)
    pre_df.loc["user"] = s_df.loc[target]
    pre_df.loc["mean"] = ms_df.mean()
    
    return pre_df

In [128]:
# test code - mean score

mean_score(s_df, sm_df, "u_1", 2)

,art_1,art_2,art_3,art_4,art_5
user,5,3,0,0,2
mean,3,0,2,3,2


In [132]:
# recommend

re_df = pre_df.T
re_df = re_df[re_df["user"]==0]
re_df = re_df.sort_values("mean", ascending=False)
print(list(re_df.index))
re_df

['art_4', 'art_3']


,user,mean
art_4,0,3
art_3,0,2


## MSE

In [141]:
def mse(value, pre):
    idx = value.nonzero()[0]
    value, pre = np.array(value)[idx], np.array(pre)[idx]
    
    return sum((value - pre) **2) / len(idx)

In [148]:
# test code mse
mse(pre_df.loc["user"], pre_df.loc["mean"])

4.333333333333333

In [145]:
def evaluate(s_df, sm_df, closer_count, algorithm):
    
    users = s_df.index
    evaluate_list = []
    
    for target in users:
        pre_df = mean_score(s_df, sm_df, target, closer_count)
        evaluate_list.append(algorithm(pre_df.loc["user"], pre_df.loc["mean"]))
    
    return np.average(evaluate_list)

In [146]:
evaluate(s_df, sm_df, 2, mse)

3.0

## RMSE

In [147]:
def rmse(value, pre):
    idx = value.nonzero()[0]
    value, pre = np.array(value)[idx], np.array(pre)[idx]
    
    return np.sqrt(sum((value - pre) **2) / len(idx))

In [149]:
# test code rmse
rmse(pre_df.loc["user"], pre_df.loc["mean"])

2.0816659994661326

In [150]:
evaluate(s_df, sm_df, 2, rmse)

1.7091905158349232

## MAE

In [157]:
def mae(value, pre):
    idx = value.nonzero()[0]
    value, pre = np.array(value)[idx], np.array(pre)[idx]
    
    return sum(np.abs(value - pre) / len(idx))

In [158]:
# test code mae
mae(pre_df.loc["user"], pre_df.loc["mean"])

1.6666666666666665

In [159]:
evaluate(s_df, sm_df, 2, mae)

1.5